# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports
from openai import OpenAI
import os
from dotenv import load_dotenv
import requests
import json
from IPython.display import Markdown, display, clear_output

In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why. Use markdowns.
yield from {book.get("author") for book in books if book.get("author")}
"""

# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

for chunk in openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that explains code."},
        {"role": "user", "content": question}
    ],
    stream=True
):
    if chunk.choices[0].delta.content:
        # Buffer to accumulate markdown output and print only on newlines
        if 'markdown_buffer' not in globals():
            markdown_buffer = ""
        markdown_buffer += chunk.choices[0].delta.content
        while "\n" in markdown_buffer:
            line, markdown_buffer = markdown_buffer.split("\n", 1)
            display(Markdown(line))
    # Print any remaining buffer at the end (outside the loop, after streaming)

Certainly! Let's break down the code snippet you provided.

### Code Explanation

```python

yield from {book.get("author") for book in books if book.get("author")}

```

...

- "Author C"

This would result in a unique set of authors, where:

- "Author A" only appears once, despite being associated with two different books.

- The book without an author (`None`) is ignored.

### Summary

In [8]:
# Get Llama 3.2 to answer
def ensure_ollama_ready():
    """Ensure Ollama is running and ready to use"""
    import subprocess
    import time
    
    # Check if Ollama is running
    try:
        response = requests.get('http://localhost:11434/api/tags', timeout=2)
        if response.status_code == 200:
            return True
    except:
        pass
    
    # Try to start Ollama
    try:
        print("�� Starting Ollama...")
        subprocess.Popen(['ollama', 'serve'], 
                        stdout=subprocess.DEVNULL, 
                        stderr=subprocess.DEVNULL)
        
        # Wait for it to be ready
        for i in range(15):
            time.sleep(1)
            try:
                response = requests.get('http://localhost:11434/api/tags', timeout=2)
                if response.status_code == 200:
                    print("✅ Ollama is ready!")
                    return True
            except:
                continue
        
        print("❌ Ollama failed to start")
        return False
        
    except FileNotFoundError:
        print("❌ Ollama not found. Please install it first.")
        return False

# Use this before your Ollama calls
ensure_ollama_ready()





# Make sure Ollama is running with: ollama run llama3.2
response = requests.post('http://localhost:11434/api/generate', json={
    'model': 'llama3.2',
    'prompt': f"""You are a helpful assistant that explains code.

User question: {question}

Please provide a clear explanation:""",
    'stream': True
})

# Buffer to collect the full response
full_response = ""

for line in response.iter_lines():
    if line:
        data = json.loads(line.decode('utf-8'))
        if 'response' in data:
            full_response += data['response']
            
            # Clear previous output and display updated markdown
            clear_output(wait=True)
            display(Markdown(full_response))
            
        if data.get('done', False):
            break

**Code Explanation**
===============

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This code uses a combination of Python features to extract authors' names from a list of books.

### Breakdown:

* `yield from`: This keyword is used to delegate the iteration over another iterable (in this case, a dictionary comprehension) to the caller.
* `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension that generates a sequence of authors' names.

### How it works:

1. The comprehension iterates over each `book` in the `books` list.
2. For each `book`, it checks if the "author" key exists and has a value using `book.get("author")`. If it does, the author's name is included in the sequence.
3. The resulting sequence of authors' names is then passed to the `yield from` statement.

### Purpose:

The purpose of this code is likely to yield each author's name as they are encountered while iterating over the list of books. This can be used in a generator function or other context where you want to process authors individually.

### Use Cases:

* Processing lists of books and extracting authors' names
* Creating an iterator that yields individual authors' names

### Example Usage:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")]

# Example list of books
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

for author in get_authors(books):
    print(author)
```

This will output:

```
Author A
None
Author C
```

In [18]:
# This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!
# If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

# Gradio UI with Text-to-Speech and Progress
import gradio as gr
import requests
import json
import openai
import base64
import tempfile
import os
import time

# List of available models
MODEL_LIST = ["llama3.2", "gpt-4o-mini"]

SYSTEM_PROMPT = """You are an expert software engineer and educator with 15+ years of experience in multiple programming languages. Your specialty is making complex code concepts accessible to learners of all levels.

**Your Approach:**
- Start with a high-level overview of what the code does
- Break down complex concepts into digestible parts
- Use analogies and real-world examples when helpful
- Explain the "why" behind design decisions, not just the "what"
- Point out potential pitfalls or edge cases
- Suggest improvements or alternative approaches when relevant

**Response Format:**
1. **Purpose**: What this code accomplishes
2. **How it works**: Step-by-step breakdown
3. **Key concepts**: Important programming concepts demonstrated
4. **Example**: Show how it would work with sample data
5. **Best practices**: Any relevant tips or warnings

Always use clear, concise language and structure your explanations with markdown formatting for readability."""

def stream_ollama_response(model, prompt, system_prompt=SYSTEM_PROMPT):
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": model,
        "prompt": f"{system_prompt}\n\nUser question: {prompt}\n\nPlease provide a clear explanation:",
        "stream": True
    }
    response = requests.post(url, headers=headers, json=data, stream=True)
    full_response = ""
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8'))
            if 'response' in data:
                full_response += data['response']
                yield full_response
            if data.get('done', False):
                break

def stream_openai_response(model, prompt, system_prompt=SYSTEM_PROMPT):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    if not openai.api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set.")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    full_response = ""
    for chunk in response:
        if hasattr(chunk, "choices") and chunk.choices:
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                full_response += delta.content
                yield full_response

def stream_response(model, prompt, system_prompt=SYSTEM_PROMPT):
    if model == "llama3.2":
        yield from stream_ollama_response(model, prompt, system_prompt)
    elif model == "gpt-4o-mini":
        yield from stream_openai_response(model, prompt, system_prompt)
    else:
        raise ValueError(f"Unknown model: {model}")

def tts_openai(text, voice="alloy"):
    """Generate TTS audio and return file path"""
    try:
        audio_response = openai.audio.speech.create(
            model="tts-1",
            voice=voice,
            input=text
        )
        
        # Save to temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            f.write(audio_response.content)
            return f.name
    except Exception as e:
        print(f"TTS Error: {e}")
        return None

def gradio_chat(user_message, history, model_choice):
    # Compose the chat history into a single prompt
    chat_prompt = ""
    for msg in history:
        if msg["role"] == "user":
            chat_prompt += f"User: {msg['content']}\n"
        elif msg["role"] == "assistant":
            chat_prompt += f"Assistant: {msg['content']}\n"
    chat_prompt += f"User: {user_message}\n"
    
    # Stream the response
    response_stream = stream_response(model_choice, chat_prompt)
    partial = ""
    for partial in response_stream:
        yield partial

def respond(user_message, history, model_choice, enable_tts, progress=gr.Progress()):
    if history is None:
        history = []
    
    # Stream the response
    response = ""
    for partial in gradio_chat(user_message, history, model_choice):
        response = partial
        current_messages = []
        for msg in history:
            current_messages.append(msg)
        current_messages.extend([
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response}
        ])
        yield current_messages, None
    
    # Generate TTS if enabled and using OpenAI
    audio_file = None
    if enable_tts and model_choice == "gpt-4o-mini":
        # Show progress for audio generation
        progress(0, desc="Starting audio generation...")
        time.sleep(0.1)  # Small delay to show progress
        
        progress(0.3, desc="Converting text to speech...")
        audio_file = tts_openai(response)
        
        progress(0.7, desc="Processing audio...")
        time.sleep(0.1)
        
        progress(1.0, desc="Audio ready!")
    
    # Final yield with complete messages
    final_messages = []
    for msg in history:
        final_messages.append(msg)
    final_messages.extend([
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": response}
    ])
    yield final_messages, audio_file

def restart_chat():
    return [], [], None

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## LLM Chat with Text-to-Speech")
    
    with gr.Row():
        model_choice = gr.Dropdown(
            choices=MODEL_LIST, 
            value=MODEL_LIST[0], 
            label="LLM Model"
        )
        enable_tts = gr.Checkbox(
            label="Enable Text-to-Speech (OpenAI only)", 
            value=False
        )
    
    chatbot = gr.Chatbot(
        label="Chat", 
        show_copy_button=True, 
        render_markdown=True,
        type="messages"
    )
    
    audio_output = gr.Audio(
        label="Generated Audio", 
        type="filepath",
        visible=True
    )
    
    with gr.Row():
        msg = gr.Textbox(label="Your message", scale=4)
        send_btn = gr.Button("Send", scale=1)
        restart_btn = gr.Button("Restart", scale=1)
    
    state = gr.State([])

    send_btn.click(
        respond,
        inputs=[msg, chatbot, model_choice, enable_tts],
        outputs=[chatbot, audio_output],
        queue=True
    )
    
    msg.submit(
        respond,
        inputs=[msg, chatbot, model_choice, enable_tts],
        outputs=[chatbot, audio_output],
        queue=True
    )

    restart_btn.click(
        restart_chat,
        inputs=None,
        outputs=[chatbot, state, audio_output],
        queue=False
    )

demo.launch()


# Tool - tbd



* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
